# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9166859029609435001
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357171200
locality {
  bus_id: 1
}
incarnation: 3320362682486988606
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    T = Tokenizer(filters='') # don't use filters, keep punctuations
    # fit on input data
    T.fit_on_texts(x)
    # transform texts to sequences
    sequences = T.texts_to_sequences(x)
    return sequences, T
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, ',': 13, 'my': 14, 'study': 15, 'of': 16, 'lexicography': 17, 'won': 18, 'prize': 19, 'this': 20, 'is': 21, 'short': 22, 'sentence': 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 5, 6, 7, 8, 2, 9, 10, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 12, 13, 14, 3, 15, 16, 17, 18, 4, 19, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [20, 21, 4, 22, 23, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    
    # get the max length of sequences
    maxlen = 0
    if length is None:
        for seq in x:
            if len(seq) > maxlen:
                maxlen = len(seq)
    else:
        maxlen = length # if length argument is given
    
    padded = pad_sequences(sequences=x, # input data
                           maxlen=maxlen, # length to pad the sequence to 
                           dtype='int32', # default data type for padding
                           padding='post', # add padding to the end
                           value=0.0) # default padding value to fill in
    return padded
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10  1]
  Output: [ 2  3  5  6  7  8  2  9 10  1  0  0]
Sequence 2 in x
  Input:  [11 12 13 14  3 15 16 17 18  4 19  1]
  Output: [11 12 13 14  3 15 16 17 18  4 19  1]
Sequence 3 in x
  Input:  [20 21  4 22 23  1]
  Output: [20 21  4 22 23  1  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 17
Max French sentence length: 23
English vocabulary size: 227
French vocabulary size: 354


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.001
    
    inputs = Input(input_shape[1:]) # input layer (first element removed because it represents N samples)
    layer1 = GRU(units=output_sequence_length,
                 return_sequences=True)(inputs) # a RNN layer with sequences as output
    
    # below is the outputs layer with vocab size
    # wrapped in TimeDistributed layer to apply Dense on each word in a sequence
    # also, softmax activation is used for multi-class classification
    outputs = TimeDistributed(Dense(units=french_vocab_size,
                                          activation='softmax'))(layer1)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 79us/step - loss: 4.5073 - acc: 0.3575 - val_loss: nan - val_acc: 0.3823
Epoch 2/10
110288/110288 [==============================] - 7s 61us/step - loss: 2.9938 - acc: 0.3813 - val_loss: nan - val_acc: 0.3823
Epoch 3/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.7605 - acc: 0.3854 - val_loss: nan - val_acc: 0.3943
Epoch 4/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.6550 - acc: 0.4075 - val_loss: nan - val_acc: 0.4260
Epoch 5/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.5879 - acc: 0.4297 - val_loss: nan - val_acc: 0.4343
Epoch 6/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.5331 - acc: 0.4382 - val_loss: nan - val_acc: 0.4395
Epoch 7/10
110288/110288 [==============================] - 7s 62us/step - loss: 2.4843 - acc: 0.4401 - val_loss: nan - val_acc: 0.44

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    
    inputs = Input(input_shape[1:]) # input layer (first element removed because it represents N samples)
    embed = Embedding(input_dim=english_vocab_size+1, # size of the vocab which starts from 0, so add 1
                      output_dim=100, # I tried a large value for embedding dim which gave good results
                      input_length=output_sequence_length)(inputs) # set the max sequence length
    rnn_gru = GRU(units=output_sequence_length,
                  return_sequences=True)(embed) # pass through an RNN layer 
    
    # outputs layer with vocab size wrapped in TimeDistributed layer
    # also, softmax activation is used for multi-class classification
    outputs = TimeDistributed(Dense(units=french_vocab_size,
                                    activation='softmax'))(rnn_gru)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model    
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length) # only pad no reshaping, embedding layer will do the job

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 74us/step - loss: 4.3611 - acc: 0.3783 - val_loss: nan - val_acc: 0.3823
Epoch 2/10
110288/110288 [==============================] - 8s 71us/step - loss: 3.0860 - acc: 0.3826 - val_loss: nan - val_acc: 0.3898
Epoch 3/10
110288/110288 [==============================] - 8s 70us/step - loss: 2.4042 - acc: 0.4726 - val_loss: nan - val_acc: 0.5473
Epoch 4/10
110288/110288 [==============================] - 8s 70us/step - loss: 1.8441 - acc: 0.5723 - val_loss: nan - val_acc: 0.6006
Epoch 5/10
110288/110288 [==============================] - 8s 71us/step - loss: 1.5556 - acc: 0.6161 - val_loss: nan - val_acc: 0.6326
Epoch 6/10
110288/110288 [==============================] - 8s 72us/step - loss: 1.3802 - acc: 0.6429 - val_loss: nan - val_acc: 0.6561
Epoch 7/10
110288/110288 [==============================] - 8s 71us/step - loss: 1.2468 - acc: 0.6690 - val_loss: nan - val_acc: 0.69

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    
    inputs = Input(input_shape[1:]) # input layer (first element removed because it represents N samples)

    # below is the RNN layer
    # wrapped in Bidirectional layer to provide reversed layers
    # By default, the layers will be concatenated
    layer1 = Bidirectional(GRU(units=output_sequence_length,
                               return_sequences=True))(inputs)
    
    # outputs layer with vocab size wrapped in TimeDistributed layer
    # also, softmax activation is used for multi-class classification
    outputs = TimeDistributed(Dense(units=french_vocab_size,
                                    activation='softmax'))(layer1)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Reshaping the input to work with a Bidirectional RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 96us/step - loss: 4.3523 - acc: 0.3674 - val_loss: nan - val_acc: 0.4323
Epoch 2/10
110288/110288 [==============================] - 10s 90us/step - loss: 2.6455 - acc: 0.4446 - val_loss: nan - val_acc: 0.4455
Epoch 3/10
110288/110288 [==============================] - 10s 90us/step - loss: 2.4197 - acc: 0.4542 - val_loss: nan - val_acc: 0.4682
Epoch 4/10
110288/110288 [==============================] - 10s 91us/step - loss: 2.2511 - acc: 0.4824 - val_loss: nan - val_acc: 0.5014
Epoch 5/10
110288/110288 [==============================] - 10s 91us/step - loss: 2.0803 - acc: 0.5168 - val_loss: nan - val_acc: 0.5342
Epoch 6/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.9312 - acc: 0.5415 - val_loss: nan - val_acc: 0.5478
Epoch 7/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.8217 - acc: 0.5486 - val_loss: nan - val_ac

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [14]:
# Refrences
# https://blog.keras.io/building-autoencoders-in-keras.html

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # OPTIONAL: Implement
    learning_rate = 0.001
    
    # encoder - passing through a 256 unit RNN layer
    inputs = Input(shape=input_shape[1:])
    encoded = GRU(256)(inputs)

    # repeat above as many as the time steps
    # to get the proper shape for the decoder
    decoded = RepeatVector(output_sequence_length)(encoded)
    decoded = GRU(french_vocab_size, return_sequences=True)(decoded)

    # outputs layer
    outputs = TimeDistributed(Dense(units=french_vocab_size,
                                    activation='softmax'))(decoded)
    
    sequence_autoencoder = Model(inputs, outputs)
    encoder = Model(inputs, encoded)

    # create a placeholder for an encoded (32-dimensional) input
    encoded_input = Input(encoded.shape)
    # retrieve the decoded layer of the autoencoder model
    # the last layer is the outputs layer(Dense) so try -2
    decoder_layer = sequence_autoencoder.layers[-2]
    # create the decoder model
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    sequence_autoencoder.compile(loss=sparse_categorical_crossentropy,
                                 optimizer=Adam(learning_rate),
                                 metrics=['accuracy'])
    
    return sequence_autoencoder

tests.test_encdec_model(encdec_model)


# TODO: Train and Print prediction(s)

# encdec model doesn't need the input to have the same shape as the output
# so reshaping without padding
tmp_x = preproc_english_sentences.reshape((-1, preproc_english_sentences.shape[-1], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 24s 219us/step - loss: 2.7221 - acc: 0.4471 - val_loss: nan - val_acc: 0.5043
Epoch 2/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.9486 - acc: 0.5286 - val_loss: nan - val_acc: 0.5459
Epoch 3/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.6581 - acc: 0.5590 - val_loss: nan - val_acc: 0.5689
Epoch 4/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.5518 - acc: 0.5729 - val_loss: nan - val_acc: 0.5799
Epoch 5/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.4732 - acc: 0.5880 - val_loss: nan - val_acc: 0.6016
Epoch 6/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.3936 - acc: 0.6045 - val_loss: nan - val_acc: 0.6043
Epoch 7/10
110288/110288 [==============================] - 23s 211us/step - loss: 1.3391 - acc: 0.6151 - val_loss: nan -

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [15]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # tried different learning rates
    # ranging from 0.001 to 0.006(included)
    # and 0.005 gave the best result
    learning_rate = 0.005
    
    # Applying embedding
    inputs = Input(input_shape[1:]) # input layer (first element removed because it represents N samples)
    embed = Embedding(input_dim=english_vocab_size+1, # size of the vocab which starts from 0, so add 1
                      output_dim=100, # embedding dimension
                      input_length=output_sequence_length)(inputs) # set the max sequence length

    # encoding layer with Bidirectional RNN
    # here, tried different number of RNN units
    # 128, 256, 512 and 1024
    # and 512 gave the best results
    encoded = Bidirectional(GRU(512))(embed)

    # decoding layers
    decoded = RepeatVector(output_sequence_length)(encoded)
    # applying Bidirectional RNN here too
    # tried applying on only one side(enc or dec) but it wasn't good enough
    decoded = Bidirectional(GRU(french_vocab_size, return_sequences=True))(decoded)
    
    # outputs layer
    outputs = TimeDistributed(Dense(units=french_vocab_size,
                                    activation='softmax'))(decoded)
    
    sequence_autoencoder = Model(inputs, outputs)
    encoder = Model(inputs, encoded)

    # create a placeholder for an encoded (32-dimensional) input
    encoded_input = Input(encoded.shape)
    # retrieve the decoded layer of the autoencoder model
    # the last layer is the outputs layer(Dense) so try -2
    decoder_layer = sequence_autoencoder.layers[-2]
    # create the decoder model
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    sequence_autoencoder.compile(loss=sparse_categorical_crossentropy,
                                 optimizer=Adam(learning_rate),
                                 metrics=['accuracy'])
    
    return sequence_autoencoder

tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

# don't need to pad(using encoder decoder model)
# and don't need to reshape(using embedding layers)
tmp_x = preproc_english_sentences

# Train the neural network
final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 76s 693us/step - loss: 2.2330 - acc: 0.4908 - val_loss: nan - val_acc: 0.5942
Epoch 2/10
110288/110288 [==============================] - 75s 677us/step - loss: 1.1253 - acc: 0.6829 - val_loss: nan - val_acc: 0.7272
Epoch 3/10
110288/110288 [==============================] - 75s 676us/step - loss: 0.7581 - acc: 0.7768 - val_loss: nan - val_acc: 0.8201
Epoch 4/10
110288/110288 [==============================] - 75s 677us/step - loss: 0.4760 - acc: 0.8587 - val_loss: nan - val_acc: 0.9015
Epoch 5/10
110288/110288 [==============================] - 75s 678us/step - loss: 0.2803 - acc: 0.9199 - val_loss: nan - val_acc: 0.9355
Epoch 6/10
110288/110288 [==============================] - 75s 681us/step - loss: 0.1790 - acc: 0.9489 - val_loss: nan - val_acc: 0.9497
Epoch 7/10
110288/110288 [==============================] - 75s 680us/step - loss: 0.1604 - acc: 0.953

## Prediction (IMPLEMENTATION)

In [16]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final  

    # get values for training
    output_len = y.shape[1]
    x_size = len(x_tk.word_index)
    y_size = len(y_tk.word_index)

    model = model_final(x.shape, 
                        output_len,
                        x_size,
                        y_size)
    
    # train
    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 77s 695us/step - loss: 2.2258 - acc: 0.4934 - val_loss: nan - val_acc: 0.6112
Epoch 2/10
110288/110288 [==============================] - 75s 679us/step - loss: 1.1012 - acc: 0.6915 - val_loss: nan - val_acc: 0.7627
Epoch 3/10
110288/110288 [==============================] - 75s 678us/step - loss: 0.7107 - acc: 0.7933 - val_loss: nan - val_acc: 0.8474
Epoch 4/10
110288/110288 [==============================] - 75s 681us/step - loss: 0.4180 - acc: 0.8779 - val_loss: nan - val_acc: 0.9159
Epoch 5/10
110288/110288 [==============================] - 75s 680us/step - loss: 0.2487 - acc: 0.9274 - val_loss: nan - val_acc: 0.9288
Epoch 6/10
110288/110288 [==============================] - 75s 677us/step - loss: 0.1678 - acc: 0.9505 - val_loss: nan - val_acc: 0.9537
Epoch 7/10
110288/110288 [==============================] - 75s 681us/step - loss: 0.1304 - acc: 0.9611 - val_loss: nan -

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [17]:
!!python -m nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 359519 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

A: I tried splitting the data into train / testset and training the model only on the trainset.
With smaller trainset, the model's loss seems to oscillate a little bit more than previous attempts but it eventually managed to learn the data pretty well.
After training, I computed the accuracy of the model on the testset which also gave pretty good score.
I also printed out some testset vs predictions sample and they also look nice.

In [18]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
    preproc_english_sentences, preproc_french_sentences, test_size=0.2)

# train the model
input_shape = X_train.shape
output_len = y_train.shape[1]
x_size = len(english_tokenizer.word_index)
y_size = len(french_tokenizer.word_index)

model = model_final(input_shape, 
                    output_len,
                    x_size,
                    y_size)

# train
model.fit(X_train, y_train, batch_size=1024, epochs=10, validation_split=0.2)

#########
# below are testing processes
predictions = model.predict(X_test)

# convert predictions into a 2d array with argmax for selecting words
convert_preds = []
for sample in predictions:
    converted = []
    for sequence in sample:
        converted.append(np.argmax(sequence))
    convert_preds.append(converted)
        
convert_preds = np.array(convert_preds)

# convert test sample into a 2d array
convert_test = y_test.reshape(y_test.shape[:-1])

# find test accuracy
test_acc = np.sum(convert_preds == convert_test) / (convert_preds.shape[0] * convert_preds.shape[1])

print('Test accuracy: {}'.format(test_acc))

# print samples
y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
y_id_to_word[0] = '<PAD>'

print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[9]]))
print(' '.join([y_id_to_word[np.max(x)] for x in y_test[9]]))

Train on 88230 samples, validate on 22058 samples
Epoch 1/10
88230/88230 [==============================] - 62s 701us/step - loss: 2.3189 - acc: 0.4858 - val_loss: 1.4456 - val_acc: 0.6104
Epoch 2/10
88230/88230 [==============================] - 60s 679us/step - loss: 1.2253 - acc: 0.6608 - val_loss: 0.9845 - val_acc: 0.7178
Epoch 3/10
88230/88230 [==============================] - 60s 678us/step - loss: 0.8504 - acc: 0.7492 - val_loss: 0.6878 - val_acc: 0.7936
Epoch 4/10
88230/88230 [==============================] - 60s 682us/step - loss: 0.5278 - acc: 0.8422 - val_loss: 0.3987 - val_acc: 0.8796
Epoch 5/10
88230/88230 [==============================] - 60s 685us/step - loss: 0.3196 - acc: 0.9059 - val_loss: 0.2590 - val_acc: 0.9245
Epoch 6/10
88230/88230 [==============================] - 60s 684us/step - loss: 0.2137 - acc: 0.9372 - val_loss: 0.1968 - val_acc: 0.9413
Epoch 7/10
88230/88230 [==============================] - 60s 683us/step - loss: 0.1963 - acc: 0.9408 - val_loss: 0.